In [1]:
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/OULAD"

import os
os.listdir(DATA_DIR)


Mounted at /content/drive


['assessments.csv',
 'studentAssessment.csv',
 'courses.csv',
 'studentRegistration.csv',
 'studentInfo.csv',
 'vle.csv',
 'studentVle.csv',
 'OULAD.names']

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [3]:
assessments = pd.read_csv(f"{DATA_DIR}/assessments.csv")
student_assessment = pd.read_csv(f"{DATA_DIR}/studentAssessment.csv")
student_vle = pd.read_csv(f"{DATA_DIR}/studentVle.csv")
student_info = pd.read_csv(f"{DATA_DIR}/studentInfo.csv")

print("assessments:", assessments.shape)
print("student_assessment:", student_assessment.shape)
print("student_vle:", student_vle.shape)
print("student_info:", student_info.shape)

assessments.head()


assessments: (206, 6)
student_assessment: (173912, 5)
student_vle: (10655280, 6)
student_info: (32593, 12)


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0


In [5]:
assessments.columns


Index(['code_module', 'code_presentation', 'id_assessment', 'assessment_type',
       'date', 'weight'],
      dtype='object')

In [6]:
ass_cols = [
    "id_assessment",
    "date",
    "assessment_type",
    "weight",
    "code_module",
    "code_presentation"
]

ass = assessments[ass_cols].rename(columns={"date": "deadline"})

df = student_assessment.merge(ass, on="id_assessment", how="left")

df = df.rename(columns={
    "id_student": "student_id",
    "id_assessment": "assessment_id",
    "date_submitted": "submit_date"
})

df.head()


,assessment_id,student_id,submit_date,is_banked,score,deadline,assessment_type,weight,code_module,code_presentation
0,1752,11391,18,0,78,19,TMA,10.0,AAA,2013J
1,1752,28400,22,0,70,19,TMA,10.0,AAA,2013J
2,1752,31604,17,0,72,19,TMA,10.0,AAA,2013J
3,1752,32885,26,0,69,19,TMA,10.0,AAA,2013J
4,1752,38053,19,0,79,19,TMA,10.0,AAA,2013J


In [8]:
import pandas as pd
import numpy as np


period_map = {
    "TMA": 14,
    "CMA": 7,
    "Exam": 30
}


df["deadline"] = pd.to_numeric(df["deadline"], errors="coerce")
df["submit_date"] = pd.to_numeric(df["submit_date"], errors="coerce")


df["period_days"] = df["assessment_type"].map(period_map).fillna(14)


df["release_date"] = df["deadline"] - df["period_days"]


df["submit_date_filled"] = df["submit_date"].fillna(df["deadline"] + 1)


df["used_time"] = df["submit_date_filled"] - df["release_date"]
df["total_period"] = df["deadline"] - df["release_date"]
df["used_ratio"] = df["used_time"] / df["total_period"]


df["missed_flag"] = df["submit_date"].isna().astype(int)

df[[
    "student_id", "assessment_id", "deadline", "submit_date",
    "release_date", "used_time", "total_period", "used_ratio", "missed_flag"
]].head(10)


,student_id,assessment_id,deadline,submit_date,release_date,used_time,total_period,used_ratio,missed_flag
0,11391,1752,19.0,18,5.0,13.0,14.0,0.928571,0
1,28400,1752,19.0,22,5.0,17.0,14.0,1.214286,0
2,31604,1752,19.0,17,5.0,12.0,14.0,0.857143,0
3,32885,1752,19.0,26,5.0,21.0,14.0,1.500000,0
4,38053,1752,19.0,19,5.0,14.0,14.0,1.000000,0
5,45462,1752,19.0,20,5.0,15.0,14.0,1.071429,0
6,45642,1752,19.0,18,5.0,13.0,14.0,0.928571,0
7,52130,1752,19.0,19,5.0,14.0,14.0,1.000000,0
8,53025,1752,19.0,9,5.0,4.0,14.0,0.285714,0
9,57506,1752,19.0,18,5.0,13.0,14.0,0.928571,0


In [9]:
df["buffer"] = df["deadline"] - df["submit_date_filled"]

def compute_rast_for_assessment(group):
    max_abs = group["buffer"].abs().max()
    if max_abs == 0:
        return pd.Series(0.0, index=group.index)
    return group["buffer"] / max_abs

df["rast"] = df.groupby("assessment_id", group_keys=False).apply(compute_rast_for_assessment)

df[["student_id","assessment_id","buffer","rast"]].head()


/tmp/ipython-input-1840553475.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df["rast"] = df.groupby("assessment_id", group_keys=False).apply(compute_rast_for_assessment)


,student_id,assessment_id,buffer,rast
0,11391,1752,1.0,0.025641
1,28400,1752,-3.0,-0.076923
2,31604,1752,2.0,0.051282
3,32885,1752,-7.0,-0.179487
4,38053,1752,0.0,0.000000


In [11]:
agg_by_student = df.groupby("student_id").agg(
    mean_used_ratio=("used_ratio", "mean"),
    std_used_ratio=("used_ratio", "std"),
    mean_rast=("rast", "mean"),
    missed_count=("missed_flag", "sum"),
    total_assessments=("assessment_id", "count")
).reset_index()

agg_by_student["missed_ratio"] = agg_by_student["missed_count"] / agg_by_student["total_assessments"]

agg_by_student.head()


,student_id,mean_used_ratio,std_used_ratio,mean_rast,missed_count,total_assessments,missed_ratio
0,6516,0.814286,0.108327,0.036858,0,5,0.0
1,8462,-1.438776,3.000405,0.454553,0,7,0.0
2,11391,0.871429,0.059761,0.061792,0,5,0.0
3,23629,1.464286,0.785714,-0.030800,0,4,0.0
4,23698,1.196429,0.316573,-0.014147,0,9,0.0


In [10]:
vle_agg = student_vle.groupby("id_student").agg(
    total_clicks=("sum_click", "sum"),
    days_active=("date", "nunique"),
    max_clicks_per_day=("sum_click", "max")
).reset_index().rename(columns={"id_student": "student_id"})

vle_agg.head()


,student_id,total_clicks,days_active,max_clicks_per_day
0,6516,2791,159,49
1,8462,656,56,16
2,11391,934,40,76
3,23629,161,16,13
4,23698,910,70,78


In [12]:
info_cols = [
    "id_student", "code_module", "code_presentation",
    "age_band", "highest_education", "imd_band",
    "num_of_prev_attempts", "final_result"
]

info = student_info[info_cols].rename(columns={"id_student": "student_id"})

features = info.merge(agg_by_student, on="student_id", how="left")
features = features.merge(vle_agg, on="student_id", how="left")

print(features.shape)
features.head()


(32593, 17)


,student_id,code_module,code_presentation,age_band,highest_education,imd_band,num_of_prev_attempts,final_result,mean_used_ratio,std_used_ratio,mean_rast,missed_count,total_assessments,missed_ratio,total_clicks,days_active,max_clicks_per_day
0,11391,AAA,2013J,55<=,HE Qualification,90-100%,0,Pass,0.871429,0.059761,0.061792,0.0,5.0,0.0,934.0,40.0,76.0
1,28400,AAA,2013J,35-55,HE Qualification,20-30%,0,Pass,1.000000,0.231455,0.013736,0.0,5.0,0.0,1435.0,80.0,23.0
2,30268,AAA,2013J,35-55,A Level or Equivalent,30-40%,0,Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,281.0,12.0,23.0
3,31604,AAA,2013J,35-55,A Level or Equivalent,50-60%,0,Pass,0.857143,0.050508,0.055948,0.0,5.0,0.0,2158.0,123.0,22.0
4,32885,AAA,2013J,0-35,Lower Than A Level,50-60%,0,Pass,1.814286,0.456249,-0.334234,0.0,5.0,0.0,1034.0,70.0,22.0


In [13]:
features["target_pass"] = features["final_result"].apply(
    lambda x: 1 if x in ["Pass","Distinction"] else 0
)

numeric_features = [
    "mean_used_ratio", "std_used_ratio", "mean_rast",
    "missed_count", "missed_ratio",
    "total_clicks", "days_active", "max_clicks_per_day",
    "num_of_prev_attempts"
]

categorical_features = [
    "age_band", "highest_education", "imd_band"
]

features[numeric_features] = features[numeric_features].fillna(0)
features[categorical_features] = features[categorical_features].fillna("Unknown")

X = features[numeric_features + categorical_features]
y = features["target_pass"]


In [14]:
# 학습 결과

numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.80      0.83      3442
           1       0.79      0.84      0.82      3077

    accuracy                           0.82      6519
   macro avg       0.82      0.82      0.82      6519
weighted avg       0.82      0.82      0.82      6519



## feature 기여도 높은 것 뽑기

In [15]:
import numpy as np

# 전처리까지 포함된 모델에서 feature importance를 꺼내려면
# model 단계 RandomForest를 직접 접근해야 함
rf = clf.named_steps["model"]
pre = clf.named_steps["preprocess"]

# OneHot + numeric 결합된 feature 이름 얻기
onehot_features = pre.named_transformers_["cat"] \
    .named_steps["onehot"].get_feature_names_out(categorical_features)

all_feature_names = np.concatenate([numeric_features, onehot_features])

# importance 추출
importances = rf.feature_importances_

# top 15만 보기
idx = np.argsort(importances)[::-1][:15]

for i in idx:
    print(f"{all_feature_names[i]}: {importances[i]:.4f}")


days_active: 0.2168
std_used_ratio: 0.2044
total_clicks: 0.1597
mean_rast: 0.1042
max_clicks_per_day: 0.1006
mean_used_ratio: 0.0995
num_of_prev_attempts: 0.0180
highest_education_Lower Than A Level: 0.0091
age_band_0-35: 0.0069
age_band_35-55: 0.0069
highest_education_A Level or Equivalent: 0.0066
imd_band_30-40%: 0.0061
imd_band_20-30%: 0.0061
imd_band_10-20: 0.0059
imd_band_50-60%: 0.0058


## XGBoost

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)

clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model)
])


## LightGBM

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(
    n_estimators=500,
    num_leaves=32,
    learning_rate=0.03
)
